In [ ]:
import numpy as np
import pandas as pd
import sklearn
import pickle

import matplotlib.pyplot as plt
import cv2

In [ ]:
#load all model
haar = cv2.CascadeClassifier('./model/haarcascade_frontalface_default.xml')
model_svm = pickle.load(open('./model/model_svm.pickle','rb'))
pca_models = pickle.load(open('./model/cpa_dict_face.pickle','rb'))

In [ ]:
model_pca = pca_models['pca']
mean_face_arr = pca_models['mean_face']

In [ ]:
# step-01: read image
img = cv2.imread('./test_HV.jpg') #RGB
#step-02: convert into gray scale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#crop the face
faces = haar.detectMultiScale(gray,1.5,3)
predictions = []
for x,y,w,h in faces:
    #cv2.rectangle(img, (x, y), (x+w, y+h), (0, 225, 0), 2)
    roi = gray[y:y+h, x:x+w]
    #nomalization(0-1)
    roi = roi/255.0
    #step-05: resize image(100x100)
    if roi.shape[1] > 100:
        roi_resize = cv2.resize(roi,(100,100),cv2.INTER_AREA)
    else:
        roi_resize = cv2.resize(roi,(100,100),cv2.INTER_CUBIC)
    #Step-06: Flattening(1x10000)
    roi_reshape = roi_resize.reshape(1,10000)
    #Step-07: Subtract with mean
    roi_mean = roi_reshape - mean_face_arr
    #Step-08: get eigen image
    eigen_image = model_pca.transform(roi_mean)
    #Step-09: eigent image for visualization
    eig_img = model_pca.inverse_transform(eigen_image)
    #Step-10: Pass to ml model (svm) and get predictions
    results = model_svm.predict(eigen_image)
    prob_score = model_svm.predict_proba(eigen_image)
    prob_score_max = prob_score.max()

    print(results,prob_score)
    # Step-11: generate report
    text = "%s: %d" % (results[0], prob_score_max * 100)
    color = (255, 0, 255)
    # Correct way to draw the rectangle
    cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
    # Use cv2.putText to add text to the image
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
    output = {
        'roi':roi,
        'eig_img':eig_img,
        'prediction_name':results[0],
        'score':prob_score_max
    }
    predictions.append(output)


In [ ]:
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10,10))
plt.imshow(img_rgb)
plt.axis('off')
plt.show()
#generate report
for i in range(len(predictions)):
    obj_gray = predictions[i]['roi'] #gray scale
    obj_eig = predictions[i]['eig_img'].reshape(100,100) # eigen image
    plt.subplot(1,2,1)
    plt.imshow(obj_gray,cmap='gray')
    plt.title('Gray ScaleImage')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(obj_eig,cmap='gray')
    plt.title('Eigen Image')
    plt.axis('off')

    plt.show()
    print('Predicted Name =',predictions[i]['prediction_name'])
    print('predicted score ={:,.2f} %'.format(predictions[i]['score']*100))
    print('-'*100)